# Hierarchical Clustering 🌳
## Building Family Trees of Data!

**Course:** Python for Machine Learning  
**Topic:** Hierarchical Clustering  
**Week:** 13 (Part 1)

---

## 📚 What You'll Learn

In this notebook, you'll learn:

1. ✅ What hierarchical clustering is and when to use it
2. ✅ The difference between agglomerative (bottom-up) and divisive (top-down)
3. ✅ How to measure distance between clusters (linkage methods)
4. ✅ How to create and interpret dendrograms
5. ✅ Implementation from scratch and with scikit-learn
6. ✅ Choosing the optimal number of clusters
7. ✅ When to use hierarchical clustering vs. K-Means

---

## 🎯 Setup: Import Libraries

Let's import everything we'll need:

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Hierarchical clustering
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import AgglomerativeClustering

# For comparison
from sklearn.cluster import KMeans

# Preprocessing
from sklearn.preprocessing import StandardScaler

# Sample datasets
from sklearn.datasets import make_blobs, make_moons

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")

---

# Part 1: Introduction to Hierarchical Clustering 🌳

## 🤔 What is Hierarchical Clustering?

Imagine organizing all living things on Earth into groups:
- **Option 1:** Start with individual animals → group similar ones → keep grouping
- **Option 2:** Start with "all animals" → split into groups → keep splitting

Both approaches create a **hierarchy** - a family tree showing relationships!

### Key Differences from K-Means:

| Feature | K-Means | Hierarchical |
|---------|---------|-------------|
| Need to specify K? | ✓ Yes | ✗ No (can decide later!) |
| Creates hierarchy? | ✗ No | ✓ Yes (dendrogram) |
| Speed | ⚡ Fast | 🐌 Slower |
| Best for | Large datasets | Understanding structure |

---

## 📊 Create Sample Data

Let's create student study habits data:

In [ ]:
# Student study data: [study_hours_per_week, test_score]
students = {
    'Alice': [2, 60],
    'Bob': [2.5, 65],
    'Charlie': [8, 90],
    'Diana': [8.5, 92],
    'Eve': [5, 75],
    'Frank': [5.5, 78],
    'Grace': [3, 68],
    'Henry': [9, 95]
}

# Convert to arrays
student_names = list(students.keys())
X = np.array(list(students.values()))

# Create DataFrame for easy viewing
df = pd.DataFrame(X, columns=['Study Hours/Week', 'Test Score'], index=student_names)
print("📚 Student Study Habits Data:")
print(df)

# Visualize
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], s=300, alpha=0.6, edgecolors='black', linewidth=2)

# Add labels
for i, name in enumerate(student_names):
    plt.annotate(name, (X[i, 0], X[i, 1]), fontsize=11, ha='center', va='bottom')

plt.xlabel('Study Hours per Week', fontsize=12)
plt.ylabel('Test Score', fontsize=12)
plt.title('Student Study Habits 📖', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

print("\n👀 Can you visually identify natural groups?")

---

# Part 2: Agglomerative Clustering (Bottom-Up) 🌱

## The Algorithm:

**Step 1:** Start with each point as its own cluster  
**Step 2:** Find the two closest clusters  
**Step 3:** Merge them into one cluster  
**Step 4:** Repeat until all points are in one cluster  

It's like building with LEGO blocks - start with individual pieces and snap them together!

---

## 💻 Build from Scratch: Simple Implementation

Let's implement a simple version to understand the process:

In [ ]:
import math

def euclidean_distance(p1, p2):
    """Calculate distance between two points"""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def cluster_distance_single(cluster1, cluster2):
    """Single linkage: minimum distance between any two points"""
    min_dist = float('inf')
    for p1 in cluster1:
        for p2 in cluster2:
            dist = euclidean_distance(p1, p2)
            min_dist = min(min_dist, dist)
    return min_dist

def simple_hierarchical_clustering(data, names):
    """Simple agglomerative clustering implementation"""
    # Start: each point is its own cluster
    clusters = [[i] for i in range(len(data))]
    cluster_data = [[data[i]] for i in range(len(data))]
    
    step = 0
    print("🌱 Starting Agglomerative Clustering (Bottom-Up)")
    print("="*60)
    print(f"\nStep {step}: {len(clusters)} clusters (each student separate)")
    for i, cluster in enumerate(clusters):
        print(f"  Cluster {i}: {[names[idx] for idx in cluster]}")
    
    # Keep merging until we have one cluster
    while len(clusters) > 1:
        step += 1
        
        # Find two closest clusters
        min_dist = float('inf')
        merge_i, merge_j = 0, 1
        
        for i in range(len(clusters)):
            for j in range(i + 1, len(clusters)):
                dist = cluster_distance_single(cluster_data[i], cluster_data[j])
                if dist < min_dist:
                    min_dist = dist
                    merge_i, merge_j = i, j
        
        # Merge the two closest clusters
        print(f"\nStep {step}: Merging clusters {merge_i} and {merge_j}")
        print(f"  Distance: {min_dist:.2f}")
        print(f"  {[names[idx] for idx in clusters[merge_i]]} + {[names[idx] for idx in clusters[merge_j]]}")
        
        # Create merged cluster
        merged_indices = clusters[merge_i] + clusters[merge_j]
        merged_data = cluster_data[merge_i] + cluster_data[merge_j]
        
        # Remove old clusters and add merged one
        new_clusters = [c for idx, c in enumerate(clusters) if idx not in [merge_i, merge_j]]
        new_data = [c for idx, c in enumerate(cluster_data) if idx not in [merge_i, merge_j]]
        
        new_clusters.append(merged_indices)
        new_data.append(merged_data)
        
        clusters = new_clusters
        cluster_data = new_data
        
        # Show current state
        print(f"  Now we have {len(clusters)} cluster(s):")
        for i, cluster in enumerate(clusters):
            print(f"    Cluster {i}: {[names[idx] for idx in cluster]}")
    
    print("\n" + "="*60)
    print("✅ Done! All students merged into one cluster!")

# Run the simple implementation
simple_hierarchical_clustering(X, student_names)

### 🎯 Key Observation:

Notice how students with similar study habits merged first:  
- Alice & Bob (both low study hours)  
- Charlie & Diana (both high study hours)  
- Eve & Frank (medium study hours)

This creates a natural hierarchy!

---

# Part 3: Linkage Methods 🔗

How do we measure distance between **clusters** (not just points)?

## Four Main Methods:

### 1. Single Linkage (Nearest Point)
Distance = **minimum** distance between any two points
- ✅ Pros: Can find chain-like patterns
- ❌ Cons: Sensitive to noise, creates long chains

### 2. Complete Linkage (Farthest Point)
Distance = **maximum** distance between any two points
- ✅ Pros: Creates compact clusters
- ❌ Cons: Sensitive to outliers

### 3. Average Linkage
Distance = **average** of all pairwise distances
- ✅ Pros: Balanced, usually works well
- ❌ Cons: Moderate computational cost

### 4. Ward's Method
Distance = increase in **variance** after merging
- ✅ Pros: Creates balanced, similar-sized clusters
- ❌ Cons: Assumes spherical clusters (like K-Means)

Let's visualize the differences!

---

## 📊 Compare All Linkage Methods

In [ ]:
# Test all linkage methods
linkage_methods = ['single', 'complete', 'average', 'ward']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, method in enumerate(linkage_methods):
    # Perform hierarchical clustering
    Z = linkage(X, method=method)
    
    # Create dendrogram
    dendrogram(Z, labels=student_names, ax=axes[idx], leaf_font_size=11)
    
    axes[idx].set_title(f'{method.capitalize()} Linkage', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Student Name', fontsize=11)
    axes[idx].set_ylabel('Distance', fontsize=11)
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.suptitle('Comparing Linkage Methods 🔗', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n🤔 Notice the differences:")
print("  • Single: May create long chains")
print("  • Complete: More compact clusters")
print("  • Average: Balanced approach")
print("  • Ward's: Most balanced, similar to K-Means")

---

# Part 4: Dendrograms 🎄

## What is a Dendrogram?

A **dendrogram** is a tree diagram that shows:
- How clusters were formed (bottom to top)
- The distance at which clusters merged (height)
- Relationships between all data points

### How to Read a Dendrogram:

```
Height = Distance at which clusters merged

    |                      ← High distance = very different
    |                    
----+----                  ← Cut here = 2 clusters
    |    |              
    |  ----+----           ← Cut here = 3 clusters
    |  |   |   |        
   [A] [B][C] [D]          ← Bottom = individual points
```

**The higher the connection, the more different the groups!**

---

## 📊 Create a Beautiful Dendrogram

In [ ]:
# Create linkage matrix using Ward's method
Z = linkage(X, method='ward')

# Create dendrogram
plt.figure(figsize=(14, 8))
dendrogram(
    Z,
    labels=student_names,
    leaf_font_size=13,
    color_threshold=15,  # Color different heights differently
)

plt.title('Student Study Habits - Hierarchical Clustering 🌳', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Student Name', fontsize=13)
plt.ylabel('Distance (How Different)', fontsize=13)

# Add cut lines
plt.axhline(y=20, color='red', linestyle='--', linewidth=2, label='Cut for 3 clusters')
plt.axhline(y=10, color='orange', linestyle='--', linewidth=2, label='Cut for 4 clusters')

plt.legend(fontsize=11)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n📖 How to read this dendrogram:")
print("  1. Bottom = individual students")
print("  2. Height = how different groups are when they merge")
print("  3. Cut horizontally to get different numbers of clusters")
print("  4. Lower connections = more similar students")

### 🎯 Interactive: Choose Number of Clusters

Unlike K-Means, we can decide K **after** seeing the dendrogram!

In [ ]:
def visualize_clusters_from_dendrogram(X, names, n_clusters, method='ward'):
    """
    Cut dendrogram at height to get n_clusters and visualize
    """
    # Perform clustering
    Z = linkage(X, method=method)
    labels = fcluster(Z, n_clusters, criterion='maxclust')
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Left: Dendrogram with cut line
    dendrogram(Z, labels=names, ax=ax1, leaf_font_size=11, color_threshold=0)
    
    # Find the height to cut for n_clusters
    if n_clusters > 1:
        # Get merge distances
        heights = Z[:, 2]
        # Find cut height for n_clusters
        cut_height = (heights[-(n_clusters-1)] + heights[-n_clusters]) / 2 if n_clusters <= len(heights) else heights[-1]
        ax1.axhline(y=cut_height, color='red', linestyle='--', linewidth=2, 
                   label=f'Cut for {n_clusters} clusters')
    
    ax1.set_title(f'Dendrogram (Cut for {n_clusters} clusters)', fontsize=13, fontweight='bold')
    ax1.set_xlabel('Student Name')
    ax1.set_ylabel('Distance')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Right: Scatter plot with cluster colors
    scatter = ax2.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', 
                         s=300, edgecolors='black', linewidth=2, alpha=0.7)
    
    # Add labels
    for i, name in enumerate(names):
        ax2.annotate(name, (X[i, 0], X[i, 1]), fontsize=10, ha='center', va='bottom')
    
    ax2.set_title(f'Resulting {n_clusters} Clusters', fontsize=13, fontweight='bold')
    ax2.set_xlabel('Study Hours per Week')
    ax2.set_ylabel('Test Score')
    ax2.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax2, label='Cluster')
    
    plt.tight_layout()
    plt.show()
    
    # Print cluster assignments
    print(f"\n📊 Cluster Assignments ({n_clusters} clusters):")
    for cluster_id in range(1, n_clusters + 1):
        members = [names[i] for i, label in enumerate(labels) if label == cluster_id]
        print(f"  Cluster {cluster_id}: {members}")

# Try different numbers of clusters
print("Let's try cutting the dendrogram at different heights:\n")

for k in [2, 3, 4]:
    print(f"\n{'='*60}")
    print(f"K = {k} clusters")
    print('='*60)
    visualize_clusters_from_dendrogram(X, student_names, k)

### 💡 Which K to Choose?

Look at the dendrogram:
- **Large vertical gaps** = good place to cut
- **Small vertical gaps** = points are similar, don't separate

For our student data:
- **K=3** seems natural: Low/Medium/High study groups
- **K=2** also works: Low vs. High performers
- **K=4** might be over-splitting

---

# Part 5: Using Scikit-learn 🚀

Now let's use the professional library!

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Create model
n_clusters = 3
model = AgglomerativeClustering(
    n_clusters=n_clusters,
    linkage='ward'  # Try: 'single', 'complete', 'average', 'ward'
)

# Fit and predict
labels = model.fit_predict(X)

print(f"🎯 Hierarchical Clustering with K={n_clusters}")
print("="*60)

# Show cluster assignments
for i, (name, label) in enumerate(zip(student_names, labels)):
    print(f"{name:10s} → Cluster {label} | Study: {X[i, 0]:.1f}h, Score: {X[i, 1]:.0f}")

# Visualize
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', 
                     s=300, edgecolors='black', linewidth=2, alpha=0.7)

# Add labels
for i, name in enumerate(student_names):
    plt.annotate(name, (X[i, 0], X[i, 1]), fontsize=11, ha='center', va='bottom')

plt.xlabel('Study Hours per Week', fontsize=12)
plt.ylabel('Test Score', fontsize=12)
plt.title(f'Hierarchical Clustering Results (K={n_clusters})', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Scikit-learn makes it easy!")

---

# Part 6: K-Means vs Hierarchical Comparison 🆚

In [ ]:
# Create non-spherical data to show differences
from sklearn.datasets import make_moons

X_moons, _ = make_moons(n_samples=200, noise=0.05, random_state=42)

# Apply both algorithms
kmeans = KMeans(n_clusters=2, random_state=42)
hierarchical = AgglomerativeClustering(n_clusters=2, linkage='average')

labels_kmeans = kmeans.fit_predict(X_moons)
labels_hierarchical = hierarchical.fit_predict(X_moons)

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# K-Means
ax1.scatter(X_moons[:, 0], X_moons[:, 1], c=labels_kmeans, cmap='viridis', 
           s=50, edgecolors='black', linewidths=0.5, alpha=0.7)
ax1.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
           c='red', marker='X', s=300, edgecolors='black', linewidths=2, label='Centroids')
ax1.set_title('K-Means Clustering', fontsize=13, fontweight='bold')
ax1.set_xlabel('Feature 1')
ax1.set_ylabel('Feature 2')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Hierarchical
ax2.scatter(X_moons[:, 0], X_moons[:, 1], c=labels_hierarchical, cmap='viridis', 
           s=50, edgecolors='black', linewidths=0.5, alpha=0.7)
ax2.set_title('Hierarchical Clustering (Average Linkage)', fontsize=13, fontweight='bold')
ax2.set_xlabel('Feature 1')
ax2.set_ylabel('Feature 2')
ax2.grid(True, alpha=0.3)

plt.suptitle('K-Means vs Hierarchical on Crescent-Shaped Data 🌙', 
            fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n🎯 Observations:")
print("  • K-Means tries to make spherical clusters (struggles here!)")
print("  • Hierarchical with average linkage handles the shape better")
print("  • Choice of linkage method matters!")

---

# Part 7: Real-World Example - Customer Segmentation 🛍️

In [ ]:
# Create synthetic customer data
np.random.seed(42)

customer_data = {
    'Customer': [f'Cust_{i:02d}' for i in range(1, 21)],
    'Annual_Income': np.random.randint(20, 120, 20),  # in $1000s
    'Spending_Score': np.random.randint(1, 100, 20)   # 1-100 scale
}

df_customers = pd.DataFrame(customer_data)

print("🛍️ Customer Segmentation Data:")
print(df_customers.head(10))
print(f"\nTotal customers: {len(df_customers)}")

# Prepare data
X_customers = df_customers[['Annual_Income', 'Spending_Score']].values

# Scale the features
scaler = StandardScaler()
X_customers_scaled = scaler.fit_transform(X_customers)

print("\n✅ Features scaled for clustering")

## Create Dendrogram to Choose K

In [ ]:
# Create linkage matrix
Z_customers = linkage(X_customers_scaled, method='ward')

# Plot dendrogram
plt.figure(figsize=(16, 8))
dendrogram(
    Z_customers,
    labels=df_customers['Customer'].values,
    leaf_font_size=10,
    color_threshold=7
)

plt.title('Customer Segmentation Dendrogram 🛍️', fontsize=15, fontweight='bold', pad=20)
plt.xlabel('Customer ID', fontsize=12)
plt.ylabel('Distance', fontsize=12)

# Add potential cut lines
plt.axhline(y=8, color='red', linestyle='--', linewidth=2, label='Cut for 3 segments')
plt.axhline(y=6, color='orange', linestyle='--', linewidth=2, label='Cut for 4 segments')

plt.legend(fontsize=11)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n💡 Based on the dendrogram, 3-4 customer segments seem natural")

## Apply Clustering and Analyze Segments

In [ ]:
# Choose K=3 based on dendrogram
n_segments = 3

# Create model
model = AgglomerativeClustering(n_clusters=n_segments, linkage='ward')
segments = model.fit_predict(X_customers_scaled)

# Add to dataframe
df_customers['Segment'] = segments

# Visualize segments
plt.figure(figsize=(12, 7))
scatter = plt.scatter(df_customers['Annual_Income'], 
                     df_customers['Spending_Score'],
                     c=segments, cmap='viridis', s=200, 
                     edgecolors='black', linewidth=1.5, alpha=0.7)

plt.xlabel('Annual Income ($1000s)', fontsize=12)
plt.ylabel('Spending Score (1-100)', fontsize=12)
plt.title(f'Customer Segmentation ({n_segments} Segments) 🎯', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Segment')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Analyze each segment
print("\n📊 Customer Segment Analysis:")
print("="*70)

for seg in range(n_segments):
    seg_data = df_customers[df_customers['Segment'] == seg]
    print(f"\n🎯 Segment {seg}:")
    print(f"  Size: {len(seg_data)} customers ({len(seg_data)/len(df_customers)*100:.1f}%)")
    print(f"  Avg Income: ${seg_data['Annual_Income'].mean():.1f}k")
    print(f"  Avg Spending Score: {seg_data['Spending_Score'].mean():.1f}")
    print(f"  Customers: {', '.join(seg_data['Customer'].values[:5])}{'...' if len(seg_data) > 5 else ''}")

print("\n" + "="*70)
print("💡 Use these segments for targeted marketing!")

---

# Part 8: Practice Exercises 🎯

## Exercise 1: Different Linkage Methods

Try all four linkage methods on the customer data. Which gives the most meaningful segments?

In [ ]:
# YOUR CODE HERE
# Try: 'single', 'complete', 'average', 'ward'



## Exercise 2: Determine Optimal K

Create a dendrogram and determine the optimal number of clusters for a new dataset:

In [ ]:
# Create new dataset
X_new, _ = make_blobs(n_samples=100, centers=4, n_features=2, 
                      cluster_std=1.0, random_state=42)

# YOUR CODE HERE
# 1. Create a dendrogram
# 2. Decide optimal K
# 3. Apply clustering
# 4. Visualize results



## Exercise 3: Compare K-Means vs Hierarchical

Apply both algorithms to the same data and compare results:

In [ ]:
# YOUR CODE HERE
# 1. Apply K-Means
# 2. Apply Hierarchical (choose linkage)
# 3. Create side-by-side visualizations
# 4. Which performs better? Why?



---

# Summary & Key Takeaways 📚

## What We Learned:

✅ **Hierarchical Clustering** creates a family tree of data  
✅ **Agglomerative** (bottom-up) is most common  
✅ **Linkage methods** determine how clusters merge  
✅ **Dendrograms** visualize the hierarchy  
✅ **No need to specify K** beforehand - decide from dendrogram  
✅ **Different from K-Means** - shows relationships, not just groups  

## When to Use Hierarchical Clustering:

✓ When you want to understand **data structure** and relationships  
✓ When you **don't know K** beforehand  
✓ When dataset is **small to medium** (<10,000 points)  
✓ When you need a **hierarchy** for interpretation  
✓ When clusters might have **different shapes** (with right linkage)  

## Linkage Method Cheat Sheet:

- **Single**: Find chain-like patterns, sensitive to noise
- **Complete**: Compact clusters, sensitive to outliers
- **Average**: Balanced, usually a safe choice
- **Ward's**: Similar to K-Means, creates balanced sizes

## Comparison:

| Feature | K-Means | Hierarchical |
|---------|---------|-------------|
| Speed | ⚡ Very Fast | 🐌 Slower |
| Specify K? | Yes | No (cut dendrogram) |
| Hierarchy? | No | Yes (dendrogram) |
| Interpretability | Good | Excellent |
| Scalability | Excellent | Poor |

---

## 🎉 Congratulations!

You now understand hierarchical clustering and can:
- Create and interpret dendrograms
- Choose appropriate linkage methods
- Determine optimal number of clusters
- Apply to real-world problems
- Compare with K-Means intelligently

**Next:** Learn about density-based clustering (DBSCAN & OPTICS)! 🎨

---

## 📚 Additional Resources

### Further Reading:
- [Scikit-learn: Hierarchical Clustering](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)
- [SciPy: Dendrograms](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html)
- Understanding different linkage methods in depth

### Practice Datasets:
- **Iris Dataset** - Classic clustering example
- **Wine Dataset** - Multi-feature hierarchical clustering
- **Customer Data** - Real-world segmentation

### Next Steps:
1. Complete all exercises in this notebook
2. Try hierarchical clustering on your own datasets
3. Experiment with different linkage methods
4. Compare results with K-Means
5. Move on to Density-Based Clustering notebook

---

**Happy Clustering! 🌳**